In [3]:
from tqdm import tqdm
from Bio import SeqIO
import re
import pandas as pd
import numpy as np
import fasttext
import random
import time

def seqToWordVec(in_fa, in_vec, model_path, word_size=10, random_choice=False,
                 rand_n=None):
    """
    Function to convert DNA sequence to a word vector representation.
    :param in_fa: Input multifasta file.
    :param in_vec: vector file generated by FastText training.
    :param word_size: Word size used to generate the corpus. Defaults to 10
    :param model_path: Path to the trained FastText model binary.
    :param random_choice: Default False. If True, will take a random sample
    == rand_n.
    :param rand_n: Number of random samples to take.
    :return: Matrix of word vectors.
    """
    print(f'Loading model:\t{model_path}\n')
    model = fasttext.load_model(model_path)
    print(f'Loading vectors:\t{in_vec}')
    word_vec = pd.read_csv(in_vec,
                           header=None,
                           index_col=0,
                           sep=' ',
                           skiprows=1)
    word_vec = word_vec.iloc[:, :30]

    word_dict = {}
    for i in tqdm(word_vec.itertuples()):
        word_dict[f'{i[0]}'] = np.reshape(np.array(list(i[1:])),
                                          newshape=(30,))

    print(f'Reading {in_fa} into dictionary  and removing N\'s')
    multi_fa = SeqIO.to_dict(SeqIO.parse(in_fa, 'fasta'))
    clean_multi_fa = {}
    for k, v in tqdm(multi_fa.items()):
        if 'N' not in str(v.seq).upper():
            clean_multi_fa[f'{k}'] = v
    if random_choice is True:
        random.seed(12)
        rand_clean_idx = random.sample(list(clean_multi_fa), k=rand_n)
        clean_fa = {key: clean_multi_fa[key] for key in rand_clean_idx}
        print(f'Number of clean records taken randomly: {rand_n}')
    else:
        clean_fa = clean_multi_fa
        print(f'Number of records before N removal: {len(multi_fa)}\nNumber of records after N removal {len(clean_fa)}')


    start_time = time.time()


    print(f'Beginning word vector represenations of {in_fa}')
    mat = []
    for k, record in enumerate(tqdm(clean_fa.items())):
        seq_arr = []
        for i in range(0, len(str(record[1].seq).upper()), word_size):
            assert str(record[1].seq).upper()[i] != 'N', 'Something went ' \
                                                         'wrong prior. You ' \
                                                         'need to make sure ' \
                                                         'there are no Ns in ' \
                                                         'the seq'

            if str(record[1].seq.upper()[i:i+word_size]) not in \
                        word_dict.keys():
                print(f'{str(record[1].seq).upper()[i:i+word_size]} not in '
                          f'dictionary. Creating vector '
                  f'from subwords.')
                seq_arr.append(model.get_word_vector(str(record[
                                                               1].seq.upper()[
                                                         i:i+word_size])))
            else:
                seq_arr.append(word_dict[str(record[1].seq).upper()[
                                       i:i+word_size]])
        seq_arr = np.array(seq_arr)
        mat.append(seq_arr.T)

    mat = np.array(mat)

    return mat

In [2]:
# Splits fasta record on N and/or n and adds it to a dictionary value for
# that chromosome. Splits fasta on N/n.
chrs = {}
for record in tqdm(SeqIO.parse
                       ('/Users/callummacphillamy/PhD/Reference_Genomes/hg38'
                        '/bt2_index/hg38.fa', 'fasta')):
    inter_seq = re.split('N|n', str(record.seq).upper())
    chrs[f'{record.name}'] = [n for n in inter_seq if n != '']



with open('./hg38_corpus.txt', 'w') as corpus:
    for ch, seq in tqdm(chrs.items()):
        for i in seq:
            for s in range(len(i)):
                corpus.write(f'{i[s:int(s+10)]} ')


455it [00:59,  7.61it/s]


In [2]:
word_vec = pd.read_csv('/Users/callummacphillamy/PhD/Reference_Genomes/hg19'
                       '/hg19.skipgram.vec',
                       header=None,
                       index_col=0,
                       sep=' ',
                       skiprows=1)
word_vec = word_vec.iloc[:, :100]


In [5]:
word_dict = {}
for i in tqdm(word_vec.itertuples()):
    word_dict[f'{i[0]}'] = np.reshape(np.array(list(i[1:])), newshape=(100,))


1048645it [00:16, 62623.50it/s]


In [6]:
enh = 'GCCGCATCCATCTCTGATGGGTGAATTAGGATCCAAATTTTTAACTAATTGAGTACATTAAATAGAAAGTCCCTAAGAAAGAAGGGAGGAAAAGACGAATACAAGACACTTTTCAAAGCTTCTTTAATAAAAAGTGTATTTGGGATTTGACCATGATGGCTATCCTTGCTTGCCAAAACCTGACGCCCTGGCAGGTAACT'
mat = []
for i in range(0,len(enh),10):
    print(enh[i:i+10])
    print(word_dict[enh[i:i+10]])
    mat.append(word_dict[enh[i:i+10]])
x = np.array(mat).T

GCCGCATCCA
[ -0.23905   -1.8646     0.5715    -0.49265   -1.9836     9.7453
   0.31082    1.5741     1.2408    -1.2396     4.7121     4.3289
  -1.3392     0.62365    0.65655   13.181    -32.464      1.2373
 -12.854     -1.176     -1.1257    -0.24657    4.0201     4.5592
   1.1543    -0.45209   -1.0573    -0.065521  -2.9976    -6.3403
   0.54074   -4.1842     0.36288    1.9754    -0.72361   -0.049955
   0.73677    1.8241    -0.50174   -2.8962    -2.5284     1.4649
  -1.5286    -1.1881    -8.5696    -4.028      1.1655    -0.93628
  -3.4753    -0.82331    1.2747    -1.921     -1.4668    -1.4764
  -0.70042    0.69487   -0.10039   -0.42633   -1.9878    -0.9687
   1.0405     2.4644     0.40661    2.5131    -0.8824     4.5574
  -4.6334     2.8202    -1.4212    -1.4906    -8.8073    -2.3719
  -0.45941    4.1874     0.86934   -2.3771     1.444     -4.7546
   3.1737    -4.2287     2.7318     3.9114    -1.309     -4.5742
   3.2415     0.54641   -2.7689    -0.11814   -3.4759     2.0328
  -1.5878  

In [1]:
def fastaToWordVec(seq, word_size):
    """
    Converts a fasta sequence to a matrix of vectors based on the a FastText
    or similar processed corpus.
    :param seq: Fasta sequence to be converted
    :param word_size: Size of the word used to generate the corpus.
    :return: A matrix of vectors that represent the fasta sequence.
    """
    mat = []
    for i in range(0, len(seq), word_size):
        if str(seq[i:i+word_size]) not in word_dict.keys():
            print(f'{seq[i:i+word_size]} not in dictionary. Creating vector '
                  f'from subwords.')
            mat.append(model.get_word_vector(str(seq[i:i+word_size])))
        else:
            mat.append(word_dict[seq[i:i+word_size]])
    mat = np.array(mat).T
    return mat

In [4]:
### POSITIVE SET ###

#from utils.utils import seqToWordVec

split = ['train','val','test']# 'val', 'test']
window = [200, 1000, 2000]
species = ['hg19', 'mm9']



for w in window:
    for s in split:
        for sp in species:
            print(f'Converting {sp} {s} at window size {w} to word-vectors.')
            path = f'../data/VISTA/{sp}/pos_fa/{sp}.VISTA.enhancers.noXY.{w}bp.{s}.fa'
            mat = seqToWordVec(path,
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.30.vec',
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.30.bin')
            print(f'Saving {sp} {s} at window size {w} to file')
            np.save(f'../data/VISTA/{sp}/pos_npy/{sp}.VISTA.enhancers.noXY.FTvec.{w}bp.{s}.npy', mat)

Converting hg19 train at window size 200 to word-vectors.
Loading model:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.30.bin

Loading vectors:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.30.vec


1048645it [00:07, 132985.70it/s]


Reading ../data/VISTA/hg19/pos_fa/hg19.VISTA.enhancers.noXY.200bp.train.fa into dictionary  and removing N's


  0%|                                                                                           | 1089/1294731 [00:00<01:58, 10877.92it/s]

Number of records before N removal: 1294731
Number of records after N removal 1294731
Beginning word vector represenations of ../data/VISTA/hg19/pos_fa/hg19.VISTA.enhancers.noXY.200bp.train.fa


...

CGTTACGCGT not in dictionary. Creating vector from subwords.


 53%|█████████████████████████████████████████████████▋                                            | 26515/50149 [00:29<00:26, 906.51it/s]

CGTTACGCGT not in dictionary. Creating vector from subwords.


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 50149/50149 [00:55<00:00, 898.10it/s]


Saving mm9 test at window size 2000 to file


In [7]:
## NEGATIVE HUMAN ##
split = ['train','val','test']# 'val', 'test']
window = [200, 1000, 2000]
species = ['hg19', 'mm9']

for w in window:
    for s in split:
        for sp in species:
            print(f'Converting {sp} {s} at window size {s} to its vector representations')
            pos = np.load(f'../data/VISTA/{sp}/pos_npy/{sp}.VISTA.enhancers.noXY'
                      f'.FTvec.{w}bp.{s}.npy',mmap_mode='r')
            mat = seqToWordVec(f'../data/VISTA/{sp}/neg_fa/{sp}.VISTA'
                           f'.non_enhancers.noXY.GCbalanced.{w}bp.{s}.fa',
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.30.vec',
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.30.bin',
                           random_choice=True,
                           rand_n=pos.shape[0])
            print(f'Saving {sp} {s} at window size {w} to file')
            np.save(f'../data/VISTA/{sp}/neg_npy/{sp}.VISTA.non_enhancers.noXY'
                f'..GCbalanced.{w}bp'
                f'.FTvec.{s}.npy', mat)

Converting hg19 train at window size train to its vector representations
Loading model:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.30.bin

Loading vectors:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.30.vec

1048645it [00:07, 145362.12it/s]


Reading ../data/VISTA/hg19/neg_fa/hg19.VISTA.non_enhancers.noXY.GCbalanced.200bp.train.fa into dictionary  and removing N's


  0%|                                                                                             | 893/1294731 [00:00<02:24, 8924.46it/s]

Number of clean records taken randomly: 1294731
Beginning word vector represenations of ../data/VISTA/hg19/neg_fa/hg19.VISTA.non_enhancers.noXY.GCbalanced.200bp.train.fa


...

ATTCGACGCG not in dictionary. Creating vector from subwords.


 41%|█████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                               | 20740/50149 [00:26<00:38, 771.90it/s]

ATTCGACGCG not in dictionary. Creating vector from subwords.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50149/50149 [01:05<00:00, 769.55it/s]


Saving mm9 test at window size 2000 to file


In [8]:
## MERGE DATASETS ##
split = ['train','val','test']# 'val', 'test']
window = [200, 1000, 2000]
species = ['hg19', 'mm9']

for w in window:
    for s in split:
        for sp in species:
            print(f'Merging {sp} {s} at window size {w} into dataset.')
            pos_X = np.load(f'../data/VISTA/{sp}/pos_npy/{sp}.VISTA.enhancers.noXY'
                            f'.FTvec.{w}bp.{s}.npy', mmap_mode='r')
            neg_X = np.load(f'../data/VISTA/{sp}/neg_npy/{sp}.VISTA.non_enhancers.noXY'
                            f'..GCbalanced.{w}bp'
                            f'.FTvec.{s}.npy', mmap_mode='r')
            pos_y = np.ones(pos_X.shape[0])
            neg_y = np.zeros(neg_X.shape[0])
            
            dat = np.vstack((pos_X, neg_X))
            lab = np.hstack((pos_y, neg_y))
            
            np.save(f'../data/VISTA/{sp}/datasets/{sp}.balanced.VISTA.FTvec.{w}bp.{s}_X.npy', dat)
            np.save(f'../data/VISTA/{sp}/datasets/{sp}.balanced.VISTA.FTvec.{w}bp.{s}_y.npy', lab)

print('Creating combined hg19 and mm9 datasets.')
for w in window:
    for s in split:
        hg19_X = np.load(f'../data/VISTA/hg19/datasets/hg19.balanced.VISTA.FTvec.{w}bp.{s}_X.npy', mmap_mode='r')
        mm9_X = np.load(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp.{s}_X.npy', mmap_mode='r')
        hg19_y = np.load(f'../data/VISTA/hg19/datasets/hg19.balanced.VISTA.FTvec.{w}bp.{s}_y.npy', mmap_mode='r')
        mm9_y = np.load(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp.{s}_y.npy', mmap_mode='r')
        
        both_dat = np.vstack((hg19_X, mm9_X))
        both_lab = np.hstack((hg19_y, mm9_y))
        
        np.save(f'../data/VISTA/hg19.mm9.balanced.VISTA.FTvec.{w}bp.{s}_X.npy', both_dat)
        np.save(f'../data/VISTA/hg19.mm9.balanced.VISTA.FTvec.{w}bp.{s}_y.npy', both_lab)
print('Finished combining hg19 and mm9 datasets.')
            
# hg19.balanced.VISTA.onehot.1000bp.test_y.npy

Merging hg19 train at window size 200 into dataset.
Merging mm9 train at window size 200 into dataset.
Merging hg19 val at window size 200 into dataset.
Merging mm9 val at window size 200 into dataset.
Merging hg19 test at window size 200 into dataset.
Merging mm9 test at window size 200 into dataset.
Merging hg19 train at window size 1000 into dataset.
Merging mm9 train at window size 1000 into dataset.
Merging hg19 val at window size 1000 into dataset.
Merging mm9 val at window size 1000 into dataset.
Merging hg19 test at window size 1000 into dataset.
Merging mm9 test at window size 1000 into dataset.
Merging hg19 train at window size 2000 into dataset.
Merging mm9 train at window size 2000 into dataset.
Merging hg19 val at window size 2000 into dataset.
Merging mm9 val at window size 2000 into dataset.
Merging hg19 test at window size 2000 into dataset.
Merging mm9 test at window size 2000 into dataset.
Creating combined hg19 and mm9 datasets.
Finished combining hg19 and mm9 datase

In [3]:
### POSITIVE SET ###
## MOUSE ##

from utils.utils import seqToWordVec

params = {'train':100000,
          'val':25000,
          'test':10000}

window = [200, 400, 1000, 1500, 2000]#, 4000, 8000]

for w in window:
    for k, v in params.items():
        print(f'Converting {w} {k} to its vector representations')
        mat = seqToWordVec(in_fa=f'../data/VISTA/mm9/pos_fa/mm9.VISTA'
                                 f'.enhancers.noXY.{w}bp.{k}.fa',
                           in_vec='/Users/callummacphillamy/Software/fastText'
                                  '-0.9.2/result/mm9.skipgram.vec',
                           model_path='/Users/callummacphillamy/Software'
                                      '/fastText-0.9.2/result/mm9.skipgram'
                                      '.bin',
                           word_size=10,
                           random_choice=True,
                           rand_n=v)
        print(f'Saving {w} {k} to file')
        np.save(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers.noXY.{w}bp'
                f'.FTvec.{k}.npy', mat)

Converting 200 train to its vector representations
Loading model:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.bin

Loading vectors:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.vec


1048724it [00:16, 62930.85it/s]


Reading ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.200bp.train.fa into dictionary  and removing N's


  1%|          | 969/100000 [00:00<00:10, 9683.22it/s]

Number of clean records taken randomly: 100000
Beginning word vector represenations of ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.200bp.train.fa


...

100%|██████████| 10000/10000 [00:31<00:00, 313.68it/s]


Saving 4000 val to file
Converting 4000 test to its vector representations
Loading model:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.bin

Loading vectors:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.vec


1048724it [00:17, 60187.60it/s]
100%|██████████| 5680/5680 [00:00<00:00, 336093.43it/s]


Reading ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.4000bp.test.fa into dictionary  and removing N's


ValueError: Sample larger than population or is negative

In [9]:
window = [4000, 8000]
split = ['train', 'val', 'test']
for w in window:
    for s in split:
        print(f'Converting {w} {s} to its vector representations')
        mat = seqToWordVec(in_fa=f'../data/VISTA/mm9/pos_fa/mm9.VISTA'
                                 f'.enhancers.noXY.{w}bp.{s}.fa',
                           in_vec='/Users/callummacphillamy/Software/fastText'
                                  '-0.9.2/result/mm9.skipgram.vec',
                           model_path='/Users/callummacphillamy/Software'
                                      '/fastText-0.9.2/result/mm9.skipgram'
                                      '.bin',
                           word_size=10)
        print(f'Saving {w} {s} to file')
        np.save(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers.noXY.{w}bp'
                f'.FTvec.{s}.npy', mat)

Converting 4000 train to its vector representations
Loading model:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.bin

Loading vectors:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.vec


1048724it [00:17, 59753.29it/s]


Reading ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.4000bp.train.fa into dictionary  and removing N's


  0%|          | 31/39754 [00:00<02:08, 308.66it/s]

Number of records before N removal: 39754
Number of records after N removal 39754
Beginning word vector represenations of ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.4000bp.train.fa


...

Saving 8000 val to file
Converting 8000 test to its vector representations
Loading model:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.bin

Loading vectors:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.vec


1048724it [00:17, 59986.56it/s]
 14%|█▎        | 9/66 [00:00<00:00, 86.88it/s]

Reading ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.8000bp.test.fa into dictionary  and removing N's
Number of records before N removal: 66
Number of records after N removal 66
Beginning word vector represenations of ../data/VISTA/mm9/pos_fa/mm9.VISTA.enhancers.noXY.8000bp.test.fa


100%|██████████| 66/66 [00:00<00:00, 87.60it/s]


Saving 8000 test to file


In [4]:
### NEGATIVE SET ###
model = fasttext.load_model('/Users/callummacphillamy/PhD/Reference_Genomes'
                            '/hg19/hg19.skipgram.bin')
split = ['train', 'val', 'test']
window = [200]#, 400, 1000, 1500, 2000, 4000, 8000]
for s in split:
    for w in window:
        print(f'Converting {w} {s} to its vector representations')
        pos = np.load(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers.noXY'
                      f'.{w}bp.FTvec.{s}.npy', mmap_mode='r')
        mat = seqToWordVec(f'../data/VISTA/mm9/neg_fa/mm9.VISTA'
                           f'.non_enhancers.noXY.GCbalanced.{w}bp.{s}.fa',
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.vec',
                           '/Users/callummacphillamy/PhD/Reference_Genomes'
                           '/hg19/hg19.skipgram.bin',
                           random_choice=True,
                           rand_n=pos.shape[0])
        print(f'Saving {w} {s} to file\n')
        np.save(f'../data/VISTA/mm9/neg_npy/mm9.VISTA.non_enhancers.noXY'
                f'.GCbalanced.{w}bp.FTvec.{s}.npy', mat)

Converting 200 train to its vector representations
Loading model:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.bin

Loading vectors:	/Users/callummacphillamy/PhD/Reference_Genomes/hg19/hg19.skipgram.vec


1048645it [00:19, 55039.58it/s]


Reading ../data/VISTA/mm9/neg_fa/mm9.VISTA.non_enhancers.noXY.GCbalanced.200bp.train.fa into dictionary  and removing N's


  0%|          | 2188/1060821 [00:00<01:35, 11096.85it/s]

Number of clean records taken randomly: 1060821
Beginning word vector represenations of ../data/VISTA/mm9/neg_fa/mm9.VISTA.non_enhancers.noXY.GCbalanced.200bp.train.fa


...

Number of clean records taken randomly: 151546
Beginning word vector represenations of ../data/VISTA/mm9/neg_fa/mm9.VISTA.non_enhancers.noXY.GCbalanced.200bp.test.fa


 41%|████      | 62102/151546 [00:06<00:13, 6450.73it/s] 

CGATCGTCGG not in dictionary. Creating vector from subwords.


100%|██████████| 151546/151546 [00:15<00:00, 9820.36it/s] 


Saving 200 test to file



In [7]:
### NEGATIVE SET ###
## MOUSE ##
split = ['train', 'val', 'test']
window = [200, 400, 1000, 1500, 2000, 4000, 8000]
for s in split:
    for w in window:
        print(f'Converting {w} {s} to its vector representations')
        pos = np.load(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers.noXY'
                      f'.{w}bp.FTvec.{s}.npy', mmap_mode='r')
        mat = seqToWordVec(f'../data/VISTA/mm9/neg_fa/mm9.VISTA'
                           f'.non_enhancers.noXY.GCbalanced.{w}bp.{s}.fa',
                           '/Users/callummacphillamy/Software/fastText-0.9'
                           '.2/result/mm9.skipgram.vec',
                           '/Users/callummacphillamy/Software/fastText-0.9'
                           '.2/result/mm9.skipgram.bin',
                           random_choice=True,
                           rand_n=pos.shape[0])
        print(f'Saving {w} {s} to file\n')
        np.save(f'../data/VISTA/mm9/neg_npy/mm9.VISTA.non_enhancers.noXY'
                f'.GCbalanced.{w}bp.FTvec.{s}.npy', mat)

Converting 200 train to its vector representations
Loading model:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.bin



Loading vectors:	/Users/callummacphillamy/Software/fastText-0.9.2/result/mm9.skipgram.vec


1048724it [00:16, 62719.52it/s]


Reading ../data/VISTA/mm9/neg_fa/mm9.VISTA.non_enhancers.noXY.GCbalanced.200bp.train.fa into dictionary  and removing N's


  2%|▏         | 2358/100000 [00:00<00:08, 11952.96it/s]

...

 39%|███▉      | 39203/100000 [00:50<01:24, 717.70it/s]

TTTCGCGACG not in dictionary. Creating vector from subwords.


100%|██████████| 100000/100000 [02:12<00:00, 754.56it/s]


Saving 2000 train to file

Converting 4000 train to its vector representations


FileNotFoundError: [Errno 2] No such file or directory: '../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers.noXY.4000bp.FTvec.train.npy'

In [5]:
### CREATE DATASET ###
import numpy as np
split = ['train', 'val', 'test']
windows = [200]
for s in split:
    for w in windows:
        print(f'Creating {s} dataset for size {w}bps')
        pos_X = np.load(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers'
                        f'.noXY.{w}bp.FTvec.{s}.npy',
                        mmap_mode='r')
        neg_X = np.load(f'../data/VISTA/mm9/neg_npy/mm9.VISTA.non_enhancers'
                        f'.noXY.GCbalanced.{w}bp.FTvec.{s}.npy', mmap_mode='r')

        pos_y = np.ones(pos_X.shape[0])
        neg_y = np.zeros(neg_X.shape[0])

        print('Stacking positive and negative arrays together.')
        dat = np.vstack((pos_X, neg_X))
        lab = np.hstack((pos_y, neg_y))
        print(f'Saving {s} data set fpr size {w}bps.')
        np.save(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp'
                f'.{s}_X'
                f'.npy', dat)
        np.save(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp'
                f'.{s}_y'
                f'.npy', lab)

Creating train dataset for size 200bps
Stacking positive and negative arrays together.
Saving train data set fpr size 200bps.
Creating val dataset for size 200bps
Stacking positive and negative arrays together.
Saving val data set fpr size 200bps.
Creating test dataset for size 200bps
Stacking positive and negative arrays together.
Saving test data set fpr size 200bps.


In [10]:
split = ['train', 'val', 'test']
windows = [200, 400, 1000, 1500, 2000, 4000, 8000]
for s in split:
    for w in windows:
        print(f'Creating dataset for size {w}bps')
        pos_X = np.load(f'../data/VISTA/mm9/pos_npy/mm9.VISTA.enhancers'
                        f'.noXY.{w}bp.FTvec.{s}.npy',
                        mmap_mode='r')
        neg_X = np.load(f'../data/VISTA/mm9/neg_npy/mm9.VISTA.non_enhancers'
                        f'.noXY'
                 f'.GCbalanced.{w}bp.FTvec.{s}.npy', mmap_mode='r')

        pos_y = np.ones(pos_X.shape[0])
        neg_y = np.zeros(neg_X.shape[0])

        dat = np.vstack((pos_X, neg_X))
        lab = np.hstack((pos_y, neg_y))
        np.save(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp'
                f'.{s}_X'
                f'.npy', dat)
        np.save(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA.FTvec.{w}bp'
                f'.{s}_y'
                f'.npy', lab)

Creating dataset for size 200bps
Creating dataset for size 400bps
Creating dataset for size 1000bps
Creating dataset for size 1500bps
Creating dataset for size 2000bps
Creating dataset for size 4000bps


FileNotFoundError: [Errno 2] No such file or directory: '../data/VISTA/mm9/neg_npy/mm9.VISTA.non_enhancers.noXY.GCbalanced.4000bp.FTvec.train.npy'

In [1]:
import numpy as np
from tqdm import tqdm
split = ['train', 'val', 'test']
window = [200]#, 1000]
for s in split:
    for w in tqdm(window):
        print(f'Combining human and mouse {s} datasets at {w}bp.')
        human_X = np.load(f'../data/VISTA/hg19/datasets/hg19.balanced.VISTA'
                          f'.FTvec.{w}bp.{s}_X.npy', mmap_mode='r')
        mouse_X = np.load(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA'
                          f'.FTvec.{w}bp.{s}_X.npy', mmap_mode='r')
        human_y = np.load(f'../data/VISTA/hg19/datasets/hg19.balanced.VISTA'
                          f'.FTvec.{w}bp.{s}_y.npy', mmap_mode='r')
        mouse_y = np.load(f'../data/VISTA/mm9/datasets/mm9.balanced.VISTA'
                          f'.FTvec.{w}bp.{s}_y.npy', mmap_mode='r')

        comb_X = np.vstack((human_X, mouse_X))
        comb_y = np.hstack((human_y, mouse_y))

        np.save(f'../data/VISTA/hg19.mm9.balanced.VISTA.FTvec.{w}bp.{s}_X'
                f'.npy', comb_X)
        np.save(f'../data/VISTA/hg19.mm9.balanced.VISTA.FTvec.{w}bp.{s}_y'
                f'.npy', comb_y)

  0%|          | 0/1 [00:00<?, ?it/s]

Combining human and mouse train datasets at 200bp.


  0%|          | 0/1 [00:00<?, ?it/s]

Combining human and mouse val datasets at 200bp.


  0%|          | 0/1 [00:00<?, ?it/s]

Combining human and mouse test datasets at 200bp.


100%|██████████| 1/1 [01:19<00:00, 79.17s/it]
